# Práctica 3: Anemometría de Hilo Caliente

Calibración (ley de King), adquisición, filtrado, estadísticos y espectro.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.signal import detrend, welch, butter, filtfilt

In [ ]:
DATA_DIR = Path('..') / 'data'
calib_path = DATA_DIR / 'calibration.csv'  # E vs Uref
signal_path = DATA_DIR / 'signal.csv'      # tiempo, voltaje
fs = 5000  # Hz ejemplo
print(calib_path, signal_path)

In [ ]:
# Cargar calibración y ajuste King: E^2 = A + B U^n
calib = pd.read_csv(calib_path)
E = calib['E'].values; Uref = calib['Uref'].values
# Ajuste log: ln(E^2 - A) ~ ln(U) -> requiere suposición inicial de A o ajuste no lineal
# Para simplicidad, ajuste polinómico en log-log como aproximación
p = np.polyfit(np.log(Uref+1e-9), np.log(E**2+1e-9), 1)
n_approx = p[0]; B_approx = np.exp(p[1])
print('n≈', n_approx, 'B≈', B_approx)

In [ ]:
# Señal y filtrado
sig = pd.read_csv(signal_path)
t = sig['t'].values; e = sig['E'].values
b,a = butter(4, 0.3, btype='low')  # normalizado a Nyquist si previamente normalizado
e_f = filtfilt(b,a, e)
e_d = detrend(e_f)
plt.figure(figsize=10,3); plt.plot(t, e, alpha=0.5, label='raw'); plt.plot(t, e_f, label='lowpass'); plt.legend(); plt.title('Señal');

In [ ]:
# Estadísticos y espectro
U_proxy = np.sqrt((e_d**2 - 0)/ (B_approx+1e-9))  # proxy simple
print('U mean ~', np.nanmean(U_proxy), 'RMS ~', np.nanstd(U_proxy))
f, Pxx = welch(e_d, fs=fs, nperseg=2048)
plt.figure(); plt.semilogx(f, Pxx); plt.title('PSD'); plt.xlabel('Hz'); plt.ylabel('PSD')